In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine("sqlite:///Chinook.sqlite")

In [3]:
table_names = engine.table_names()

In [4]:
print(table_names)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [5]:
con = engine.connect()
rs = con.execute("SELECT * FROM Artist")
df = pd.DataFrame(rs.fetchall())
con.close()

In [6]:
print(df.head(10))  #default is first 4

    0                     1
0   1                 AC/DC
1   2                Accept
2   3             Aerosmith
3   4     Alanis Morissette
4   5       Alice In Chains
5   6  Antonio Carlos Jobim
6   7          Apocalyptica
7   8            Audioslave
8   9              BackBeat
9  10          Billy Cobham


In [7]:
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Artist")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
print(df.head())

   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains


In [8]:
df1 = pd.read_sql_query("SELECT * FROM Artist", engine)

In [9]:
print(df1)

     ArtistId                                               Name
0           1                                              AC/DC
1           2                                             Accept
2           3                                          Aerosmith
3           4                                  Alanis Morissette
4           5                                    Alice In Chains
5           6                               Antonio Carlos Jobim
6           7                                       Apocalyptica
7           8                                         Audioslave
8           9                                           BackBeat
9          10                                       Billy Cobham
10         11                                Black Label Society
11         12                                      Black Sabbath
12         13                                         Body Count
13         14                                    Bruce Dickinson
14         15            

In [10]:
print(df.equals(df1))

True


In [11]:
from sqlalchemy import MetaData, Table

In [12]:
MD = MetaData()
TB = Table("Employee", MD, autoload = True, autoload_with = engine)
# Read database and build SQLalchemy Table objects

In [13]:
print(repr(TB))
# Get all Col names at Employee table
# repr() function to print all detail of Employee Table

Table('Employee', MetaData(bind=None), Column('EmployeeId', INTEGER(), table=<Employee>, primary_key=True, nullable=False), Column('LastName', NVARCHAR(length=20), table=<Employee>, nullable=False), Column('FirstName', NVARCHAR(length=20), table=<Employee>, nullable=False), Column('Title', NVARCHAR(length=30), table=<Employee>), Column('ReportsTo', INTEGER(), ForeignKey('Employee.EmployeeId'), table=<Employee>), Column('BirthDate', DATETIME(), table=<Employee>), Column('HireDate', DATETIME(), table=<Employee>), Column('Address', NVARCHAR(length=70), table=<Employee>), Column('City', NVARCHAR(length=40), table=<Employee>), Column('State', NVARCHAR(length=40), table=<Employee>), Column('Country', NVARCHAR(length=40), table=<Employee>), Column('PostalCode', NVARCHAR(length=10), table=<Employee>), Column('Phone', NVARCHAR(length=24), table=<Employee>), Column('Fax', NVARCHAR(length=24), table=<Employee>), Column('Email', NVARCHAR(length=60), table=<Employee>), schema=None)


In [14]:
print(TB.columns.keys())
# only get columns names of table

['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email']


In [15]:
print(repr(MD.tables["Employee"]))
# The other method to get all detail

Table('Employee', MetaData(bind=None), Column('EmployeeId', INTEGER(), table=<Employee>, primary_key=True, nullable=False), Column('LastName', NVARCHAR(length=20), table=<Employee>, nullable=False), Column('FirstName', NVARCHAR(length=20), table=<Employee>, nullable=False), Column('Title', NVARCHAR(length=30), table=<Employee>), Column('ReportsTo', INTEGER(), ForeignKey('Employee.EmployeeId'), table=<Employee>), Column('BirthDate', DATETIME(), table=<Employee>), Column('HireDate', DATETIME(), table=<Employee>), Column('Address', NVARCHAR(length=70), table=<Employee>), Column('City', NVARCHAR(length=40), table=<Employee>), Column('State', NVARCHAR(length=40), table=<Employee>), Column('Country', NVARCHAR(length=40), table=<Employee>), Column('PostalCode', NVARCHAR(length=10), table=<Employee>), Column('Phone', NVARCHAR(length=24), table=<Employee>), Column('Fax', NVARCHAR(length=24), table=<Employee>), Column('Email', NVARCHAR(length=60), table=<Employee>), schema=None)


In [16]:
TB1 = Table("Customer", MD, autoload = True, autoload_with = engine)
print(TB1.columns.keys())

['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']


In [17]:
TB2 = Table("Invoice", MD, autoload = True, autoload_with = engine)
print(TB2.columns.keys())

['InvoiceId', 'CustomerId', 'InvoiceDate', 'BillingAddress', 'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode', 'Total']


In [28]:
# Found out Table Customer and Invoice should be linked of CustomerId
df2 = pd.read_sql_query("SELECT InvoiceId, InvoiceDate,City, SupportRepId ,Total FROM Customer INNER JOIN Invoice on Customer.CustomerId = Invoice.CustomerId",engine)
print(df2.head())

# Col City and SupportRepId are from Customer Table
# Col InvoiceId, InvoiceDate and Total are from Invoice Table

   InvoiceId          InvoiceDate       City  SupportRepId  Total
0          1  2007-01-01 00:00:00  Stuttgart             5   1.98
1          2  2007-01-02 00:00:00       Oslo             4   3.96
2          3  2007-01-03 00:00:00   Brussels             4   5.94
3          4  2007-01-06 00:00:00   Edmonton             5   8.91
4          5  2007-01-11 00:00:00     Boston             4  13.86


In [21]:
df2 = pd.read_sql_query("SELECT * FROM PlaylistTrack INNER JOIN Track on PlaylistTrack.TrackId = Track.TrackId WHERE Milliseconds < 250000", engine)
# another way to extract all records where Milliseconds < 250000 of 2 tables

In [22]:
print(df2.head())

   PlaylistId  TrackId  TrackId              Name  AlbumId  MediaTypeId  \
0           1     3390     3390  One and the Same      271            2   
1           1     3392     3392     Until We Fall      271            2   
2           1     3393     3393     Original Fire      271            2   
3           1     3394     3394       Broken City      271            2   
4           1     3395     3395          Somedays      271            2   

   GenreId Composer  Milliseconds    Bytes  UnitPrice  
0       23     None        217732  3559040       0.99  
1       23     None        230758  3766605       0.99  
2       23     None        218916  3577821       0.99  
3       23     None        228366  3728955       0.99  
4       23     None        213831  3497176       0.99  
